In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#common libs
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
from torchinfo import summary
import math
from easydict import EasyDict as edict


In [3]:
#mtr modules
from mtr.datasets import build_dataloader
from mtr.config import cfg, cfg_from_yaml_file
from mtr.utils import common_utils

In [4]:
cfg_from_yaml_file("/code/jjiang23/csc587/KimchiVision/cfg/kimchiConfig.yaml", cfg)
logger = common_utils.create_logger("/files/waymo/damon_log.txt", rank=0)
args = edict({
    "batch_size": 1,
    "workers": 32,
    "merge_all_iters_to_one_epoch": False,
    "epochs": 5,
    "add_worker_init_fn": False,
    
})

In [5]:
#prepare data
train_set, train_loader, train_sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    batch_size=args.batch_size,
    dist=False, workers=args.workers,
    logger=logger,
    training=True,
    merge_all_iters_to_one_epoch=args.merge_all_iters_to_one_epoch,
    total_epochs=args.epochs,
    add_worker_init_fn=args.add_worker_init_fn,
)

test_set, test_loader, sampler = build_dataloader(
        dataset_cfg=cfg.DATA_CONFIG,
        batch_size=args.batch_size,
        dist=False, workers=args.workers, logger=logger, training=False
)

2025-06-08 08:55:33,032   INFO  Start to load infos from /files/waymo/code/MTR/data/waymo/processed_scenarios_training_infos.pkl


2025-06-08 08:55:39,146   INFO  Total scenes before filters: 243401
2025-06-08 08:55:45,576   INFO  Total scenes after filter_info_by_object_type: 243401
2025-06-08 08:55:45,585   INFO  Total scenes after filters: 243401
2025-06-08 08:55:45,587   INFO  Start to load infos from /files/waymo/code/MTR/data/waymo/processed_scenarios_val_infos.pkl
2025-06-08 08:55:47,924   INFO  Total scenes before filters: 22089
2025-06-08 08:55:48,514   INFO  Total scenes after filter_info_by_object_type: 22089
2025-06-08 08:55:48,544   INFO  Total scenes after filters: 22089


In [ ]:
from lstm.lstm import MotionLSTM
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MotionLSTM().to(device)

home of the LSTM /home/dlin42/KimchiVision, change when needed


In [7]:
batch = next(iter(train_loader))

home of the LSTM /home/dlin42/KimchiVision, change when needed


Traceback (most recent call last):
  File "/files/waymo/miniconda3/envs/lstm/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/files/waymo/miniconda3/envs/lstm/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/files/waymo/miniconda3/envs/lstm/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/files/waymo/miniconda3/envs/lstm/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


In [8]:
batch["input_dict"].keys()

dict_keys(['scenario_id', 'obj_trajs', 'obj_trajs_mask', 'track_index_to_predict', 'obj_trajs_pos', 'obj_trajs_last_pos', 'obj_types', 'obj_ids', 'center_objects_world', 'center_objects_id', 'center_objects_type', 'obj_trajs_future_state', 'obj_trajs_future_mask', 'center_gt_trajs', 'center_gt_trajs_mask', 'center_gt_final_valid_idx', 'center_gt_trajs_src', 'map_polylines', 'map_polylines_mask', 'map_polylines_center', 'static_map_polylines', 'static_map_polylines_mask'])

In [9]:
input = batch["input_dict"]
obj_trajs = input["obj_trajs"]
obj_pos = input["obj_trajs_pos"]
obj_last_pos = input["obj_trajs_last_pos"]
obj_type = input["obj_types"] # car, bicycycle, pedestrian
obj_trajs_mask = input['obj_trajs_mask']
obj_of_interest = input['track_index_to_predict']

In [10]:
num_center_objects, num_objects, num_timestamps, num_attrs = obj_trajs.shape

In [13]:
static_map_polylines=input["static_map_polylines"].to('cuda')  # (batch_size, num_polylines, num_points_each_polyline, 7)
static_map_polylines_mask=input["static_map_polylines_mask"].to('cuda') # (batch_size, num_polylines, num_points_each_polyline)

In [42]:
obj_trajs_mask[:, 1, :].sum(dim=1)

tensor([11, 11, 11, 11, 11, 11, 11, 11])

In [80]:
from einops import rearrange, repeat

torch.nonzero(obj_trajs_mask[0, 0, :])[-1]

tensor([10])

In [19]:
model._print_batch(batch)

home of the LSTM /home/dlin42/KimchiVision, change when needed
Key: scenario_id, Val: (8,)
Key: obj_trajs, Val: torch.Size([8, 24, 11, 29])
Key: obj_trajs_mask, Val: torch.Size([8, 24, 11])
Key: track_index_to_predict, Val: torch.Size([8])
Key: obj_trajs_pos, Val: torch.Size([8, 24, 11, 3])
Key: obj_trajs_last_pos, Val: torch.Size([8, 24, 3])
Key: obj_types, Val: (24,)
Key: obj_ids, Val: (24,)
Key: center_objects_world, Val: torch.Size([8, 10])
Key: center_objects_id, Val: (8,)
Key: center_objects_type, Val: (8,)
Key: obj_trajs_future_state, Val: torch.Size([8, 24, 80, 4])
Key: obj_trajs_future_mask, Val: torch.Size([8, 24, 80])
Key: center_gt_trajs, Val: torch.Size([8, 80, 4])
Key: center_gt_trajs_mask, Val: torch.Size([8, 80])
Key: center_gt_final_valid_idx, Val: torch.Size([8])
Key: center_gt_trajs_src, Val: torch.Size([8, 91, 10])
Key: map_polylines, Val: torch.Size([8, 768, 20, 9])
Key: map_polylines_mask, Val: torch.Size([8, 768, 20])
Key: map_polylines_center, Val: torch.Size([8

In [103]:
map_object_features = model(batch)


In [106]:
#define loss function
class MotionLoss(nn.Module):
    """
    Loss function for trajectory prediction
    """
    
    def __init__(self, 
                 regression_loss_weight=1.0,
                 classification_loss_weight=1.0,
                 future_loss_weight=1.0):
        super(MotionLoss, self).__init__()
        self.reg_weight = regression_loss_weight
        self.cls_weight = classification_loss_weight
        self.future_weight = future_loss_weight
    
    def forward(self, pred_scores, pred_trajs, batch_dict):
        """
        Compute loss
        
        Args:
            pred_scores: (batch_size, num_modes)
            pred_trajs: (batch_size, num_modes, future_steps, 4)
            batch_dict: Contains ground truth data
        
        Returns:
            loss_dict: Dictionary containing different loss components
        """
        center_gt_trajs = batch_dict['input_dict']['center_gt_trajs'].to('cuda')  # (batch_size, future_steps, 4)
        center_gt_trajs_mask = batch_dict['input_dict']['center_gt_trajs_mask'].to('cuda')  # (batch_size, future_steps)
        
        batch_size, num_modes, future_steps, _ = pred_trajs.shape
        
        # Compute trajectory regression loss for each mode
        gt_trajs_expanded = center_gt_trajs.unsqueeze(1).expand(-1, num_modes, -1, -1)
        gt_mask_expanded = center_gt_trajs_mask.unsqueeze(1).expand(-1, num_modes, -1)
        
        # L2 loss for position (x, y)
        pos_loss = F.mse_loss(
            pred_trajs[:, :, :, :2] * gt_mask_expanded.unsqueeze(-1),
            gt_trajs_expanded[:, :, :, :2] * gt_mask_expanded.unsqueeze(-1),
            reduction='none'
        ).sum(dim=-1)  # (batch_size, num_modes, future_steps)
        
        # L2 loss for velocity (vx, vy)
        vel_loss = F.mse_loss(
            pred_trajs[:, :, :, 2:4] * gt_mask_expanded.unsqueeze(-1),
            gt_trajs_expanded[:, :, :, 2:4] * gt_mask_expanded.unsqueeze(-1),
            reduction='none'
        ).sum(dim=-1)  # (batch_size, num_modes, future_steps)
        
        # Weighted loss over time (give more weight to near future)
        time_weights = torch.exp(-0.1 * torch.arange(future_steps, device=pred_trajs.device))
        time_weights = time_weights.view(1, 1, -1)
        
        pos_loss = (pos_loss * time_weights * gt_mask_expanded).sum(dim=-1)  # (batch_size, num_modes)
        vel_loss = (vel_loss * time_weights * gt_mask_expanded).sum(dim=-1)  # (batch_size, num_modes)
        
        # Find best mode for each sample
        total_traj_loss = pos_loss + vel_loss  # (batch_size, num_modes)
        best_mode_indices = torch.argmin(total_traj_loss, dim=1)  # (batch_size,)
        
        # Regression loss (best mode)
        best_pos_loss = pos_loss[torch.arange(batch_size), best_mode_indices].mean()
        best_vel_loss = vel_loss[torch.arange(batch_size), best_mode_indices].mean()
        regression_loss = best_pos_loss + best_vel_loss
        
        # Classification loss (encourage higher confidence for best mode)
        target_scores = torch.zeros_like(pred_scores)
        target_scores[torch.arange(batch_size), best_mode_indices] = 1.0
        classification_loss = F.cross_entropy(pred_scores, target_scores)
        
        # Total loss
        total_loss = (self.reg_weight * regression_loss + 
                     self.cls_weight * classification_loss)
        
        loss_dict = {
            'total_loss': total_loss,
            'regression_loss': regression_loss,
            'classification_loss': classification_loss,
            'pos_loss': best_pos_loss,
            'vel_loss': best_vel_loss
        }
        
        return loss_dict


In [107]:
#train loop
def train_model(model, train_dataloader, val_dataloader, num_epochs=100, lr=1e-3):
    """
    Training loop for the LSTM model
    """
    assert torch.cuda.is_available(), "CUDA is not available. Please check your PyTorch installation."
    device = torch.device('cuda')
    model.to(device)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    criterion = MotionLoss()
    
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_losses = []
        
        for batch_idx, batch_dict in enumerate(train_dataloader):
            # Move data to device
            for key, value in batch_dict.items():
                if isinstance(value, torch.Tensor):
                    batch_dict[key] = value.to(device)
            
            optimizer.zero_grad()
            
            # Forward pass
            pred_scores, pred_trajs = model(batch_dict)
            
            # Compute loss
            loss_dict = criterion(pred_scores, pred_trajs, batch_dict)
            loss = loss_dict['total_loss']
            
            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_losses.append(loss.item())
            
            if batch_idx % 100 == 0:
                print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item():.4f}')
        
        # Validation phase
        model.eval()
        val_losses = []
        
        with torch.no_grad():
            for batch_dict in val_dataloader:
                # Move data to device
                for key, value in batch_dict.items():
                    if isinstance(value, torch.Tensor):
                        batch_dict[key] = value.to(device)
                
                pred_scores, pred_trajs = model(batch_dict)
                loss_dict = criterion(pred_scores, pred_trajs, batch_dict)
                val_losses.append(loss_dict['total_loss'].item())
        
        scheduler.step()
        
        avg_train_loss = np.mean(train_losses)
        avg_val_loss = np.mean(val_losses)
        
        print(f'Epoch {epoch}: Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')
        
        # Save best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), '/code/jjiang23/csc587/KimchiVision/best_motion_lstm.pth')
    
    return model

In [110]:
from lstm.lstm import MotionLSTM
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MotionLSTM().to(device)

# Train the model
trained_model = train_model(model, train_loader, test_loader)

Epoch 0, Batch 0, Loss: 12214.0684
Epoch 0, Batch 100, Loss: 260.8756
Epoch 0, Batch 200, Loss: 304.0185
Epoch 0, Batch 300, Loss: 40.1907
Epoch 0, Batch 400, Loss: 178.3487
Epoch 0, Batch 500, Loss: 164.0680
Epoch 0, Batch 600, Loss: 114.9102
Epoch 0, Batch 700, Loss: 807.5247
Epoch 0, Batch 800, Loss: 111.6026
Epoch 0, Batch 900, Loss: 946.8884
Epoch 0, Batch 1000, Loss: 95.1272
Epoch 0, Batch 1100, Loss: 150.1637
Epoch 0, Batch 1200, Loss: 302.3832
Epoch 0, Batch 1300, Loss: 122.2616
Epoch 0, Batch 1400, Loss: 132.4782
Epoch 0, Batch 1500, Loss: 100.0318
Epoch 0, Batch 1600, Loss: 85.4091
Epoch 0, Batch 1700, Loss: 245.8979
Epoch 0, Batch 1800, Loss: 232.2650
Epoch 0, Batch 1900, Loss: 115.5724
Epoch 0, Batch 2000, Loss: 93.3215
Epoch 0, Batch 2100, Loss: 23.0922
Epoch 0, Batch 2200, Loss: 143.0038
Epoch 0, Batch 2300, Loss: 101.1137
Epoch 0, Batch 2400, Loss: 165.7377
Epoch 0, Batch 2500, Loss: 47.1470
Epoch 0, Batch 2600, Loss: 69.6619
Epoch 0, Batch 2700, Loss: 118.7284
Epoch 0, 

KeyboardInterrupt: 